In [2]:
import pyspark 
import argparse 
import json 

# 4.1.1 D: Top m users who have the largest number of reviews and its count

sc_conf = pyspark.SparkConf()
sc_conf.setAppName('task1')  
sc_conf.setMaster('local[*]') 
sc_conf.set('spark.driver.memory', '8g')
sc_conf.set('spark.executor.memory', '4g')
# sc = pyspark.SparkContext(conf=sc_conf)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel("OFF")

# read in the lines and create the rdd 
lines = sc.textFile("./data/review.json")

# loading the json into a python object 
rdd = lines.map(lambda line: json.loads(line))

your 131072x1 screen size is bogus. expect trouble


23/02/03 12:56:10 WARN Utils: Your hostname, LAPTOP-0S5927HR resolves to a loopback address: 127.0.1.1; using 192.168.250.233 instead (on interface eth0)
23/02/03 12:56:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 12:56:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/03 12:56:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/03 12:56:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [15]:
m = 10 
unique_users = rdd.map(lambda x: (x['user_id'])).map(lambda user: (user, 1)).reduceByKey(lambda a, b: a+b).sortBy(lambda x: x[1], False).take(m)
print(unique_users)



[('CxDOIDnH8gp9KXzpBHJYXw', 715), ('bLbSNkLggFnqwNNzzq-Ijw', 424), ('PKEzKWv_FktMm2mGPjwd0Q', 322), ('DK57YibC5ShBmqQl97CKog', 291), ('ELcQDlf69kb-ihJfxZyL0A', 288), ('U4INQZOPSUaj8hMjLlZ3KA', 276), ('QJI9OSEn6ujRCtrX06vs1w', 258), ('d_TBs6J3twMy9GChqUEXkg', 253), ('hWDybu_KvYLSdEFzGrniTw', 239), ('dIIKEfOgo0KqUfGQvGikPg', 216)]


In [ ]:
print(unique_users)

In [13]:
sol = []
m = 5 
for i in range(m):
    sol.append(unique_users[i])
print(sol)

[('CxDOIDnH8gp9KXzpBHJYXw', 715), ('bLbSNkLggFnqwNNzzq-Ijw', 424), ('PKEzKWv_FktMm2mGPjwd0Q', 322), ('DK57YibC5ShBmqQl97CKog', 291), ('ELcQDlf69kb-ihJfxZyL0A', 288)]


[('BTxxGYdw8CNV7UZGc9YU1Q', <pyspark.resultiterable.ResultIterable object at 0x7ff4c3357940>), ('KeBt3UtPDr7GbQsHx9pJNw', <pyspark.resultiterable.ResultIterable object at 0x7ff4c33543d0>)]


In [ ]:
# at this point how do I aggregate the lists associated with each key 
# do I use a combineByKey() somehow? 


In [ ]:
# how do I then sort by the max value for the keys 

[('wLofLemhls9oQqFpc4qEQQ', (1, 'gmfckUvQZQmcPm8RGRiwOQ')),
 ('Ml5Q1rLjyUmS-Q2dQ2C4nw', (1, '-i7iRw_DI_T0wTSXsfiY8Q')),
 ('dEk9YPjigYm8_7f9H8t_EQ', (1, 'byNMn9Hf9cv_Amj2_5ydow')),
 ('b9-9XmpCefla_r-6-Dm0vg', (1, 'YvqDqcjAvCvs-H1IU7viLg'))]

<class 'pyspark.rdd.PipelinedRDD'>


In [ ]:
grouping = unique_reviews.groupByKey()

In [ ]:
grouping = grouping.map(lambda x, y: (x, len(y)))
print(type(grouping))

In [ ]:
grouping.collect()